In [1]:
import sys
!{sys.executable} -m pip install gate_api

In [26]:
import requests
import pandas as pd
from datetime import datetime

host = "https://api.gateio.ws"
prefix = "/api/v4"
headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}

In [55]:
# get all currency pairs
url = '/spot/currencies'
r = requests.request('GET', host + prefix + url, headers=headers)
r.json()

[{'currency': 'CREDIT',
  'delisted': False,
  'withdraw_disabled': False,
  'withdraw_delayed': False,
  'deposit_disabled': False,
  'trade_disabled': False},
 {'currency': 'STPT',
  'delisted': False,
  'withdraw_disabled': False,
  'withdraw_delayed': False,
  'deposit_disabled': False,
  'trade_disabled': False},
 {'currency': 'UGT',
  'delisted': True,
  'withdraw_disabled': False,
  'withdraw_delayed': False,
  'deposit_disabled': False,
  'trade_disabled': False},
 {'currency': 'GRIN3L',
  'delisted': False,
  'withdraw_disabled': True,
  'withdraw_delayed': False,
  'deposit_disabled': True,
  'trade_disabled': False},
 {'currency': 'RENBTC',
  'delisted': False,
  'withdraw_disabled': False,
  'withdraw_delayed': False,
  'deposit_disabled': False,
  'trade_disabled': False},
 {'currency': 'QTUM_ETH',
  'delisted': False,
  'withdraw_disabled': False,
  'withdraw_delayed': False,
  'deposit_disabled': False,
  'trade_disabled': False},
 {'currency': 'BBK',
  'delisted': True,

In [59]:
# get currency pair info
url = '/spot/currencies/GT'
query_param = 'currency_pair=DOGE_USDT'
r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
r.json()

{'currency': 'GT',
 'delisted': False,
 'withdraw_disabled': False,
 'withdraw_delayed': False,
 'deposit_disabled': False,
 'trade_disabled': False}

### Historical Data (candelsticks)
+ limited to 1000 per pull.  Can download more to csv from the website
+ https://www.gate.io/en/developer/historical_quotes
+ https://www.gate.io/docs/developers/apiv4/en/#market-candlesticks

In [50]:
# candlesticks = historical data by interval.  Limited to 1000
url = '/spot/candlesticks'
pair = 'BTC_USDT'
interval = '1d'
query_param = 'currency_pair='+pair+'&'+'interval='+interval
cols = ['timestamp','volume','close','high','low','open']

# get, convert to df
r = requests.request('GET', host + prefix + url + "?" + query_param, headers=headers)
df = pd.DataFrame(list(r.json()),columns = cols)

# add the currency pair, convert unix timestamp to date and set as the index
df['pair'] = pair
df['date'] = df.timestamp.apply(lambda x:datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
df.set_index('date',inplace=True)
df.drop(['timestamp'],axis=1,inplace=True)

df.head()

,volume,close,high,low,open,pair
date,,,,,,
2021-05-16 00:00:00,155000727.331137982848047,46434.14,49777.4,43890,46791.01,BTC_USDT
2021-05-17 00:00:00,192324747.872776226163361,43550.75,46613.25,42101,46434.93,BTC_USDT
2021-05-18 00:00:00,139869239.552834866366068,42849.89,45782.63,42291.13,43547.39,BTC_USDT
2021-05-19 00:00:00,489307863.858910411065444107,36662.15,43591.94,29800,42854.18,BTC_USDT
2021-05-20 00:00:00,321546290.995153450628666,40548.02,42409.35,34908.82,36669.73,BTC_USDT


In [67]:
# historical download of data
# hourly data by month- requirs a download per month
btc = pd.read_csv('../data/BTC_USDT.csv',header=None,names=cols)
btc['pair'] = 'BTC_USDT'
btc['date'] = btc.timestamp.apply(lambda x:datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
btc.set_index('date',inplace=True)
btc.drop(['timestamp'],axis=1,inplace=True)
btc

,volume,close,high,low,open,pair
date,,,,,,
2018-01-31 23:00:00,14.227681,10313.077462,10314.058075,10150.849999,10203.319998,BTC_USDT
2018-01-31 22:00:00,5.991346,10194.550147,10288.156528,10127.269877,10146.486278,BTC_USDT
2018-01-31 21:00:00,3.364490,10105.750000,10199.960041,10001.000000,10178.329998,BTC_USDT
2018-01-31 20:00:00,5.144728,10177.590000,10210.001155,10062.730000,10076.110000,BTC_USDT
2018-01-31 19:00:00,7.451171,10069.100000,10200.000000,10030.399999,10098.993436,BTC_USDT
...,...,...,...,...,...,...
2018-01-01 04:00:00,17.580909,13499.000551,13500.015726,13103.299999,13289.000000,BTC_USDT
2018-01-01 03:00:00,29.545778,13289.000000,13504.756757,13200.130000,13248.279157,BTC_USDT
2018-01-01 02:00:00,25.614961,13248.279157,13386.881606,13112.580000,13308.950000,BTC_USDT
